Requires you to have run the timing experiment `jobscripts/memory_efficiency/time_test_all.sh`

In [1]:
import os

logging_dir = '/work3/s184399/msc/logs/'
result_dirs = list(filter(lambda x: ('time_test' in x) and (len(os.listdir(os.path.join(logging_dir, x))) > 0), os.listdir(logging_dir)))
result_files = list([list(filter(lambda x: 'time_test' in x, os.listdir(os.path.join(logging_dir, d)))) for d in result_dirs])
result_files = list(map(lambda x: x[0], result_files))

In [2]:
import numpy as np

def bootstrap_CI(p, alpha=0.05, k=2000, n_decimals=0):
  """
    Computes the confidence interval of the mean using bootstrapping.
    Here the confidence interval is the 100*(1-alpha) central CI, from percentile 100*(alpha/2) to 100*(1-alpha/2) rounded to broadest interval when picking the indices.
    Line Clemmensen suggests picking k (number of repeats) to 1000 or 2000 for this tasks, so I do this.
  """
  assert isinstance(p, np.ndarray)
  assert p.ndim == 1
  N = len(p)
  bootstraps = np.random.choice(p, (k,N), replace=True)
  ci_lower = alpha/2.
  ci_upper = 1.-(alpha/2.)
  idxs = [
    int(np.floor(k*ci_lower)),
    int(np.ceil(k*ci_upper))
  ]
  CI = np.sort(np.mean(bootstraps, axis=-1))[idxs]     # Sorts lowest to highest
  assert CI[0] < CI[1]  # To be on the safe side...
  CI = [f"{c:.{n_decimals}f}s" for c in CI]
  return CI, N    # Returns CI and support (N)

In [4]:
import numpy as np
import matplotlib.pyplot as plt

for i in range(len(result_dirs)):
    times = np.loadtxt(os.path.join(logging_dir, result_dirs[i], result_files[i]), delimiter=',', skiprows=1)
    print(f"{result_files[i].replace('time_test_','').replace('.txt','')}: {bootstrap_CI(times, n_decimals=2)[0]}")

opt-125m_adamw_torch: ['3.02s', '3.14s']
opt-125m_MEBP_adamw_torch: ['1.35s', '1.44s']
Llama-2-7b-hf_sgd: ['36.97s', '37.52s']
opt-125m_QLoRA_adamw_torch: ['0.99s', '1.00s']
opt-125m_MEBP_sgd: ['1.37s', '1.40s']
Llama-2-7b-hf_QLoRA_adamw_torch: ['5.79s', '5.91s']
Llama-2-7b-hf_MEBP_sgd: ['36.72s', '37.37s']
opt-125m_sgd: ['1.33s', '1.38s']


In [5]:
import numpy as np
import matplotlib.pyplot as plt

for i in range(len(result_dirs)):
    times = np.loadtxt(os.path.join(logging_dir, result_dirs[i], result_files[i]), delimiter=',', skiprows=1)
    print(f"{result_files[i].replace('time_test_','').replace('.txt','')}: {np.mean(times)}")

opt-125m_adamw_torch: 3.0701682712092544
opt-125m_MEBP_adamw_torch: 1.388477238741788
Llama-2-7b-hf_sgd: 37.23242569210554
opt-125m_QLoRA_adamw_torch: 0.9911088269166272
opt-125m_MEBP_sgd: 1.3811171849568684
Llama-2-7b-hf_QLoRA_adamw_torch: 5.847253741640033
Llama-2-7b-hf_MEBP_sgd: 37.032277165037215
opt-125m_sgd: 1.3525888173267095
